# Metrics Across Validate Projects

The purpose of this notebook is to explore the existing population of MapSwipe Validate projects. It highlights some interesting facts about the user and project populations and provides initial framing of some features we might use to set remapping thresholds for an individual project's building footprints.

In [ ]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

In [ ]:
import seaborn as sns
sns.set(style='white',font_scale=1.0,rc={"axes.spines.top":False,"axes.spines.right":False, "lines.linewidth": 2.5,'lines.markersize': 10},color_codes=False,palette=sns.color_palette(['#27a3aa','#f76d23','#70d6e3','#ffbb31','#b1c96d','#cce18a','#1c4c5d','#787642']))

# Metrics for all projects

## Calculate the metrics

There are 74 projects scoped for this analysis out of 88 total validate projects:
* 6 not finished yet (inactive or active)
* 1 whose geoms were squares instead of building footprints - https://download.geoservice.dlr.de/WSF2019/
* 7 whose data couldn't be downloaded

In [ ]:
from mapswipe.data_access import read_scoped_projects_list, get_project_data
df_projects = read_scoped_projects_list()
validate_projects = list(df_projects["project_id"])
df_projects[["project_type", "status"]].value_counts().sort_index()

In [ ]:
import diskcache
from mapswipe.data_access import CACHE_PATH, CACHE_SIZE
with diskcache.Cache(directory=CACHE_PATH, size_limit=CACHE_SIZE) as cache:
    all_proj_data = cache["all_proj_data"]

In [ ]:
list(all_proj_data.keys())[50]

In [ ]:
#project_id = "-MxuKEABaIRO1bvsDGpM"
project_id = "-Mv4SEfo6NwzCqy6YGby"
data = all_proj_data[project_id]

In [ ]:
df_projects[df_projects["project_id"] == project_id]

In [ ]:
def has_offset(row):
    return any(all_proj_data[row["project_id"]]["agg"]["3_count"])

df_projects["has_offset"] = df_projects.apply(has_offset, axis=1)
df_projects["has_offset"].value_counts()

`custom_options` always shows offset as an option in the project, even for projects where nobody has used it.

In [ ]:
df_projects[["has_offset", "custom_options"]].value_counts()

## User Stats

In [ ]:
df_full_all = pd.concat([all_proj_data[project_id]["full"] for project_id in validate_projects])

### Who is the validate userbase?

In [ ]:
df_user_proj = df_full_all.drop_duplicates().groupby("user_id").agg(
    involved_project_count=("project_id", "nunique"),
    first_seen=("timestamp", "min"),
    last_seen=("timestamp", "max"),
)
df_user_proj["first_seen"] = pd.to_datetime(df_user_proj["first_seen"], format="mixed").dt.floor("min")
df_user_proj["last_seen"] = pd.to_datetime(df_user_proj["last_seen"], format="mixed").dt.floor("min")
df_user_proj["tenure_days"] = (df_user_proj["last_seen"] - df_user_proj["first_seen"]).apply(lambda x: x.days + 1)
df_user_proj["user_weight"] = df_user_proj["involved_project_count"].apply(lambda x: max(1, np.log2(x)))

Almost 27,000 users have contributed to validate projects

In [ ]:
df_user_proj[["involved_project_count", "tenure_days"]].describe()

In [ ]:
df_user_proj.head()

The userbase follows a power law distribution - a small number of very involved power users, and a large number of users who worked on a single project for one day

In [ ]:
df_user_proj.reset_index().sort_values("involved_project_count", ascending=False).plot(x="user_id", y="involved_project_count").get_xaxis().set_visible(False)

In [ ]:
(df_user_proj
 .sort_values("involved_project_count", ascending=False)
 .reset_index()
 .rename({"involved_project_count": "# Contributed Projects"}, axis=1)
).head()

In [ ]:
(df_user_proj
 .sort_values("involved_project_count", ascending=False)
 .reset_index()
 .rename({"involved_project_count": "# Contributed Projects"}, axis=1)
 .plot(y="# Contributed Projects")
 .set_xlabel("Users")
 #.get_xaxis().set_visible(False))
)

In [ ]:
df_user_proj.reset_index().sort_values("tenure_days", ascending=False).plot(x="user_id", y="tenure_days").get_xaxis().set_visible(False)

#### How do they typically respond?

In [ ]:
df_full_all.head()

In [ ]:
df_user_responses = df_full_all.groupby("user_id").apply(lambda x: x["result"].value_counts()).reset_index().pivot(columns="result", index="user_id", values="count").fillna(0.0)
df_user_responses = df_user_responses.rename_axis(None, axis=1).stack().rename({i: f"{i}_count_all" for i in range(4)}).unstack()
df_user_responses["total_count_all"] = df_user_responses[[f"{i}_count_all" for i in range(4)]].sum(axis=1)
for i in range(4):
    df_user_responses[f"{i}_share_all"] = df_user_responses[f"{i}_count_all"] / df_user_responses["total_count_all"]

In [ ]:
df_user_responses.sort_values("total_count_all").tail(10)

On average, users say:
* No - 50%
* Yes - 35%
* Not Sure - 5%
* Offset - 1%

But there are some users whose responses consist entirely of one answer!

In [ ]:
df_user_responses.boxplot(column=[f"{i}_share_all" for i in range(4)], rot=90, figsize=(10, 5))

In [ ]:
response_cols = {
    "0_share_all": "No",
    "1_share_all": "Yes",
    "2_share_all": "Not Sure",
    "3_share_all": "Offset",
}

(df_user_responses
 .rename(response_cols, axis=1)
 .boxplot(column=list(response_cols.values()), rot=90, figsize=(10, 5))
)
#df_user_responses.rename(response_cols, axis=1).head()

Out of almost 27k users, 1236 of them have only ever given one type of response.

In [ ]:
from functools import reduce
from operator import or_

len(df_user_responses), len(df_user_responses[reduce(or_, [df_user_responses[f"{i}_share_all"] == 1.0 for i in range(4)])])

### Power users

These are your most engaged users.

**Policy question - should these users' contributions have more weight than less involved users?**

In [ ]:
df_user_proj[df_user_proj["involved_project_count"] > 20]

### Project-level user involvement

In [ ]:
df_proj_user = df_full_all.drop_duplicates().groupby("project_id").agg(
    user_count=("user_id", "nunique"),
    first_seen=("timestamp", "min"),
    last_seen=("timestamp", "max"),
)
df_proj_user["duration_days"] = (pd.to_datetime(df_proj_user["last_seen"], format="mixed") - pd.to_datetime(df_proj_user["first_seen"], format="mixed")).apply(lambda x: x.days + 1)

These figures summarize projects across number of users and project length (time between first and last user response).

In [ ]:
df_proj_user.describe()

In [ ]:
df_proj_user.plot.scatter(x="user_count", y="duration_days")

### Num users vs num involved projects per project

In [ ]:
df_project_users = df_full_all[["project_id", "user_id"]].drop_duplicates()
df_project_users = df_project_users.merge(df_user_proj[["involved_project_count"]].reset_index(), on="user_id", how="inner")
df_project_users = df_project_users.groupby("project_id").agg({"user_id": "nunique", "involved_project_count": "median"})
df_project_users = df_project_users.rename({"user_id": "num_users", "involved_project_count": "median_involved_project_count"}, axis=1)

In [ ]:
df_proj_year = df_projects[["project_id", "created"]].drop_duplicates()
df_proj_year["created"] = pd.to_datetime(df_proj_year["created"])
df_proj_year["project_year"] = df_proj_year["created"].dt.strftime("%Y").astype(int)
df_proj_year = df_proj_year.drop("created", axis=1)

In [ ]:
df_project_users = df_project_users.merge(df_proj_year, on="project_id", how="inner")

In [ ]:
(df_project_users
 .plot
 .scatter(x="num_users", y="median_involved_project_count", c="project_year", cmap="Set1")
)
plt.grid(visible=True)
plt.xlabel("# users in project")
plt.ylabel("Median contributed projects across project users")
plt.title("Per-project users vs # contributed projects")
plt.show()

## Recalculate agg stats with user weights

In [ ]:
df_full_all.head()

In [ ]:
df_full_all_weight = df_full_all.merge(df_user_proj["user_weight"].reset_index(), on="user_id", how="inner")
len(df_full_all), len(df_full_all_weight)

In [ ]:
df_full_all_weight.head()

In [ ]:
df_agg_weight = df_full_all_weight.groupby(["project_id", "task_id", "result"]).agg({"user_weight": "sum"}).round()

In [ ]:
df_agg_weight_share = df_agg_weight.reset_index().pivot_table(values="user_weight", index=["project_id", "task_id"], columns="result", fill_value=0.0)
df_agg_weight_share.head()

In [ ]:
w_count_cols = [f"{i}_count" for i in df_agg_weight_share.columns]
df_agg_weight_share.columns = w_count_cols
df_agg_weight_share["total_count"] = df_agg_weight_share[w_count_cols].sum(axis=1)
for i in range(4):
    df_agg_weight_share[f"{i}_share"] = df_agg_weight_share[f"{i}_count"] / df_agg_weight_share["total_count"]

In [ ]:
df_agg_weight_share.head()

## Task-Level Stats

In [ ]:
all_proj_data[validate_projects[50]]["agg"].head()

In [ ]:
def agg_all(project_id, df):
    df["project_id"] = project_id
    return df

df_agg_all = pd.concat([agg_all(project_id, all_proj_data[project_id]["agg"]) for project_id in validate_projects if all_proj_data[project_id]["agg"] is not None])
df_agg_all = df_agg_all.drop("idx", axis=1).set_index(["project_id", "task_id"])

df_full_user = df_full_all.groupby(["project_id", "task_id"]).agg(
    user_count=("user_id", "nunique"),
    first_seen=("timestamp", "min"),
    last_seen=("timestamp", "max"),
)
df_agg_all = df_agg_all.join(df_full_user).reset_index()
del df_full_user


import h3

def to_h3(row, resolution):
    rp = row.geometry.representative_point()
    #return h3.geo_to_h3(row.geometry.y, row.geometry.x, resolution)
    try:
        return h3.geo_to_h3(rp.y, rp.x, resolution)
    except:
        return None

df_agg_all["h3_hex"] = df_agg_all.apply(to_h3, axis=1, resolution=8)
df_h3_count = df_agg_all[["h3_hex", "task_id"]].groupby("h3_hex").nunique().rename({"task_id": "h3_building_count"}, axis=1).reset_index()
df_agg_all = df_agg_all.merge(df_h3_count, on="h3_hex")

# TODO fine-tune this
df_agg_all["is_urban"] = (df_agg_all["h3_building_count"] >= 150)

In [ ]:
df_agg_all.head()

In [ ]:
df_agg_all = df_agg_all.rename((
    {f"{i}_count": f"{i}_count_uw" for i in range(4)}
    | {f"{i}_share": f"{i}_share_uw" for i in range(4)}
    | {"total_count": "total_count_uw", "incorrect_score": "incorrect_score_uw"}
), axis=1)

In [ ]:
df_agg_all = df_agg_all.set_index(["project_id", "task_id"]).join(df_agg_weight_share, how="left").reset_index()
# TODO improve this logic beyond the yes share
df_agg_all["incorrect_score"] = 1 - df_agg_all["1_share"]

In [ ]:
df_agg_all.head()

In [ ]:
df_agg_all["incorrect_score"].describe()

This is an initial pass at determining urban vs rural - at least 150 buildings in a 0.7 km^2 H3 hexagon (resolution 8). Given that definition, this shows how many buildings fall in the same hexagon.

We're still working on this definition, for example because it misrepresents large buildings (more on that later).

In [ ]:
df_h3_count.describe()

### Task Basics

In [ ]:
len(df_agg_all)

This is the distribution of users who have seen an individual task

In [ ]:
with pd.option_context("display.float_format", lambda x: "%.3f" % x):
    print(df_agg_all["user_count"].describe())

Even within a given project, there's more variation in the number of users who see a task than we thought we'd see.

In [ ]:
df_task_users = df_agg_all[["project_id", "user_count"]].groupby("project_id").agg(
    user_count_min=("user_count", "min"),
    user_count_max=("user_count", "max"),
    user_count_mean=("user_count", "mean"),
)
df_task_users["user_count_range"] = df_task_users["user_count_max"] - df_task_users["user_count_min"]
df_task_users.sample(10, random_state=50)

In [ ]:
df_task_users[["user_count_range"]].describe()

By our evolving definition of "urban", most of the tasks describe buildings in urban settings.

In [ ]:
df_agg_all["is_urban"].describe()

Here are summary statistics about the different features we're looking at.

In [ ]:
with pd.option_context("display.float_format", lambda x: "%.3f" % x):
    print(df_agg_all[["incorrect_score", "nearby_building_count", "building_area_m2", "user_count"]].describe())

### Correlation across metrics

The sections below demonstrate a few points:
* Different projects have wildly different levels of correlation between features and the target variable, so it's harder to draw conclusions across all projects
* Intuitively, larger buildings have more agreement and less uncertainty in user responses

In [ ]:
from scipy.stats import pearsonr, spearmanr

# TODO REMOVE FACETING FOR URBAN/RURAL

def calc_corr_by_project(df_input, corr_cols, target_col, facet_cols=None):
    def _calc_corr_grp(grp):
        ret = []
        for corr_col in corr_cols:
            p = pearsonr(grp[corr_col], grp[target_col])
            s = spearmanr(grp[corr_col], grp[target_col])
            ret.append({
                "col1_name": corr_col,
                "col2_name": target_col,
                "pearson_stat": p.statistic,
                "pearson_pval": p.pvalue,
                "spearman_stat": s.statistic,
                "spearman_pval": s.pvalue,
            })
        return pd.DataFrame(ret).set_index("col1_name")

    key_cols = ["project_id"] + (facet_cols if facet_cols else [])
    return df_input.groupby(key_cols).apply(_calc_corr_grp).reset_index()


def calc_corr_for_slides(df_input, corr_cols, corr_col_descs, target_col, facet_cols=None):
    def _calc_corr_grp(grp):
        ret = []
        for corr_col in corr_cols:
            p = pearsonr(grp[corr_col], grp[target_col])
            s = spearmanr(grp[corr_col], grp[target_col])
            ret.append({
                "col1_name": corr_col,
                "col2_name": target_col,
                "pearson_stat": p.statistic,
                "pearson_pval": p.pvalue,
                "spearman_stat": s.statistic,
                "spearman_pval": s.pvalue,
            })
        return pd.DataFrame(ret).set_index("col1_name")

    key_cols = ["project_id"] + (facet_cols if facet_cols else [])
    df = df_input.groupby(key_cols).apply(_calc_corr_grp).reset_index()

    df = df[(df["pearson_pval"] <= 0.05) & (df["spearman_pval"] <= 0.05)]
    df["stat"] = df[["pearson_stat", "spearman_stat"]].mean(axis=1)
    df["col1_name"] = df["col1_name"].replace({corr_cols[i]: corr_col_descs[i] for i in range(len(corr_cols))})
    df = df.rename({"col1_name": "Feature Name"}, axis=1)
    return df


import matplotlib.pyplot as plt

def plot_corr(df_input, target_variable):
    df_input[["col1_name", "is_urban", "pearson_stat", "spearman_stat"]].boxplot(by=["col1_name", "is_urban"], rot=90, figsize=(10, 5))
    plt.suptitle(f"Project-level correlation to {target_variable}")
    plt.show()

def plot_corr_slides(df_input, target_variable, target_var_desc):
    df_input[["Feature Name", "stat"]].boxplot(by="Feature Name", figsize=(10, 6))
    plt.suptitle(f"Project-level Feature Correlation to {target_var_desc}")
    plt.title("")
    plt.xlabel("")
    plt.ylabel("Correlation")
    plt.show()

#### Correlation to incorrect_score

In [ ]:
df_corr_score_slide = calc_corr_for_slides(
    df_agg_all, 
    ["h3_building_count", "building_area_m2", "user_count"],
    ["Nearby Building Count", "Building Footprint Size", "# Task Responses"],
    "incorrect_score", 
)

In [ ]:
df_corr_score_slide.head(10)

In [ ]:
plot_corr_slides(df_corr_score_slide, "incorrect_score", "Incorrectness Score")

In [ ]:
df_corr_score = calc_corr_by_project(
    df_agg_all, 
    ["nearby_building_count", "h3_building_count", "building_area_m2", "user_count"], 
    "incorrect_score", 
    facet_cols=["is_urban"]
)
df_corr_score = df_corr_score[(df_corr_score["pearson_pval"] <= 0.05) & (df_corr_score["spearman_pval"] <= 0.05)]

In [ ]:
len(df_corr_score)

In [ ]:
plot_corr(df_corr_score, "incorrect_score")

#### Correlation to agreement

In [ ]:
df_corr_agree_slide = calc_corr_for_slides(
    df_agg_all, 
    ["h3_building_count", "building_area_m2", "user_count"],
    ["Nearby Building Count", "Building Footprint Size", "# Task Responses"],
    "agreement", 
)

In [ ]:
plot_corr_slides(df_corr_agree_slide, "agreement", "Agreement Score")

In [ ]:
df_corr_agree = calc_corr_by_project(
    df_agg_all, 
    ["nearby_building_count", "h3_building_count", "building_area_m2", "user_count"], 
    "agreement", 
    facet_cols=["is_urban"]
)
df_corr_agree = df_corr_agree[(df_corr_agree["pearson_pval"] <= 0.05) & (df_corr_agree["spearman_pval"] <= 0.05)]

In [ ]:
len(df_corr_agree)

In [ ]:
plot_corr(df_corr_agree, "agreement")

#### Correlation to 2_share ("not sure") + offset

There doesn't seem to be much difference in correlation between projects with and without offset.

In [ ]:
offset_projects = set(df_projects[df_projects["has_offset"]]["project_id"])

##### Projects with "offset" as a choice

In [ ]:
df_corr_unsure_offset = calc_corr_by_project(
    df_agg_all[df_agg_all["project_id"].isin(offset_projects)], 
    ["nearby_building_count", "h3_building_count", "building_area_m2", "user_count"], 
    "2_share", 
    facet_cols=["is_urban"]
)
df_corr_unsure_offset = df_corr_unsure_offset[(df_corr_unsure_offset["pearson_pval"] <= 0.05) & (df_corr_unsure_offset["spearman_pval"] <= 0.05)]

In [ ]:
len(df_corr_unsure_offset)

In [ ]:
plot_corr(df_corr_unsure_offset, "2_share")

##### Projects without "offset" as a choice

In [ ]:
df_corr_unsure_nooffset = calc_corr_by_project(
    df_agg_all[~(df_agg_all["project_id"].isin(offset_projects))], 
    ["nearby_building_count", "h3_building_count", "building_area_m2", "user_count"], 
    "2_share", 
    facet_cols=["is_urban"]
)
df_corr_unsure_nooffset = df_corr_unsure_nooffset[(df_corr_unsure_nooffset["pearson_pval"] <= 0.05) & (df_corr_unsure_nooffset["spearman_pval"] <= 0.05)]

In [ ]:
len(df_corr_unsure_nooffset)

In [ ]:
plot_corr(df_corr_unsure_nooffset, "2_share")

# Outlier Exploration

In [ ]:
import folium
import branca.colormap as cm

def create_task_map(gdf, center_pt=None, color_col="1_share"):
    
    geojson_data = gdf.drop('lastEdit', axis=1).to_json()

    if center_pt is None:
        center_pt = gdf.to_crs(gdf.estimate_utm_crs()).dissolve().centroid.to_crs(4326)
    map = folium.Map(location=[center_pt.y, center_pt.x], zoom_start=8)
    map._repr_html_ = lambda: map._parent._repr_html_(
    include_link=False, width='75%', height='400px'
    )

    colormap = cm.linear.YlOrRd_09.scale(gdf[color_col].min(), gdf[color_col].max())

    def style_function(feature):
        return {
            'fillColor': colormap(feature['properties'][color_col]),
            'color': 'black',
            'weight': 0.5,
            'fillOpacity': 0.8
        }

    

    folium.GeoJson(
        geojson_data,
        style_function=style_function,
        name="geojson"
    ).add_to(map)

    colormap.add_to(map)

    return map

## Projects with many nearby buildings

In [ ]:
df_agg_all[df_agg_all["nearby_building_count"] >= 500]["project_id"].drop_duplicates().head()

In [ ]:
df_projects[df_projects["project_id"].isin(df_agg_all[df_agg_all["nearby_building_count"] >= 500]["project_id"].drop_duplicates().head())]

## Projects with large buildings

In [ ]:
df_agg_all[df_agg_all["building_area_m2"] > 10000.0][["project_id", "building_area_m2"]].groupby("project_id").sum().sort_values("building_area_m2")

In [ ]:
project_id = "-NEaR6DbJAbkpYJ_BDCH"
#create_task_map(all_proj_data[project_id]["agg"])
create_task_map(df_agg_all[df_agg_all["project_id"] == project_id].replace({"is_urban": {True: 1.0, False: 0.0}}), color_col="is_urban")

In [ ]:
df_agg_all[df_agg_all["project_id"] == project_id]["h3_hex"].value_counts().describe()